In [1]:
import torch
print(torch.__version__)

2.4.1+cu118


In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [8]:
!pip install timm einops

                                              0.0/2.3 MB ? eta -:--:--
     ------                                   0.4/2.3 MB 11.8 MB/s eta 0:00:01
     ------------                             0.7/2.3 MB 9.3 MB/s eta 0:00:01
     ----------------                         0.9/2.3 MB 7.4 MB/s eta 0:00:01
     ------------------                       1.1/2.3 MB 6.3 MB/s eta 0:00:01
     ---------------------                    1.3/2.3 MB 5.8 MB/s eta 0:00:01
     -------------------------                1.5/2.3 MB 5.4 MB/s eta 0:00:01
     ----------------------------             1.6/2.3 MB 5.2 MB/s eta 0:00:01
     -------------------------------          1.8/2.3 MB 5.0 MB/s eta 0:00:01
     ----------------------------------       2.0/2.3 MB 4.9 MB/s eta 0:00:01
     -------------------------------------    2.2/2.3 MB 4.8 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 4.8 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 4.5 MB/s eta 


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from transformers import AutoProcessor, AutoModelForCausalLM

b:\Playground\LNRS\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model = AutoModelForCausalLM.from_pretrained(r'B:\Playground\LNRS\florence_docvqa-transformers-default-v1\Florence2-DocVQA',trust_remote_code=True).to("cuda").eval()
processor = AutoProcessor.from_pretrained(r'B:\Playground\LNRS\florence_docvqa-transformers-default-v1\Florence2-DocVQA', trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/HuggingFaceM4/Florence-2-DocVQA:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [11]:
def run_example(task_prompt, image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_answer

def process_image(image, text_input=None):
    image = Image.open(image)
    image=image.convert("RGB")
    task_prompt = 'What is'
    results = run_example(task_prompt, image, text_input)[task_prompt].replace("<pad>", "")
    return results


def noice(img_path,ques):
    try:
        wrong=process_image(img_path,ques)
        if wrong=='unanswerable':
            return ''
        return wrong
    
    except Exception as e:
        
        print(f"{e}")
        return ''

In [13]:
img_path=r'B:\Playground\LNRS\loan2.png'
q="what is date?"
ans=noice(img_path,q)
ans

C:\Users\acer\.cache\huggingface\modules\transformers_modules\HuggingFaceM4\Florence-2-DocVQA\c49996de96f172540d619ddfc5ee1241098a2740\modeling_florence2.py:1209: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'The date of the loan agreement is October, 2024.'